## 0. Imports

In [ ]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack # -> for multiple environments
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import ale_py
import os

In [ ]:
# !pip install gymnasium[atari]

## 1. Test env

In [ ]:
env_name = "ALE/Breakout-v5"
env = gym.make(env_name, render_mode="human")

In [ ]:
env.reset()

In [ ]:
env.action_space

In [ ]:
env.observation_space

In [ ]:
# (must be run every time)  # simple random-play loop to see env dynamics
env = gym.make(env_name, render_mode="human")

episodes = 5
for episode in range(1, episodes+1):
    # gymnasium may return (obs, info) in newer API; simple cases sometimes return just obs
    state = env.reset()
    terminated = False
    truncated = False
    done = False
    score = 0

    while not done:
        env.render()
        # random action (useful to inspect action space)
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated  # Set 'done' for the while loop condition
        score += reward

    print('Episode:{} Score:{}'.format(episode, score))

env.close()

In [ ]:
env.close()

## 2. Vectorize env and train model

In [ ]:
env = make_atari_env(env_name, n_envs=4, seed=0) # -> make vector
env = VecFrameStack(env, n_stack=4) # -> stack them

In [ ]:
model = A2C("CnnPolicy", env, verbose=1)

In [ ]:
model.learn(total_timesteps=100000)

## 3. Save model

In [ ]:
a2c_path = os.path.join('../Training', 'Saved_Models', 'A2C_Breakout_model_100k')

In [ ]:
model.save(a2c_path)

In [ ]:
model = A2C.load(a2c_path)

## 4. Evaluate and test model

In [ ]:
env = make_atari_env(env_name, n_envs=1, seed=0) # -> make vector
env = VecFrameStack(env, n_stack=4) # -> stack them
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [ ]:
env.close()